# Gas Network Problem

There is a network of nodes where some gas is either produced or consumed. The goal of the problem is to find the best way to transport the gas from the production nodes to the consumption nodes.

You have to decide whether a pipeline is constructed between two nodes or not. A pipeline has a capacity and a cost to be opened (i.e, constructed). Once the pipeline is opened, you also have a cost for each unit of gas that is transported through this pipeline. Finally, there is a penalty for each unit of produced gas which is not consumed.

#### You want to minimize the total cost.

In [1]:
# consumption (demand)
d = [ 0, 50, 95, 10, 73, 55, 125, 32, 40, 20 ]
# production (maximum generation)
#gmax = [ 500, 0, 0, 500, 0, 0, 500, 0, 0, 0 ]
gmax = [ 500, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
N = len(d)
NODES = range(N)
print('generation - demand? (%d): %d' % (N, sum(gmax) - sum(d)))

generation - demand? (10): 0


In [2]:
# capacity of the arcs
ca = [ [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
       [20, 30, 40, 50, 60, 70, 80, 90, 100, 10],
       [30, 40, 50, 60, 70, 80, 90, 100, 10, 20],
       [40, 50, 60, 70, 80, 90, 100, 10, 20, 30],
       [50, 60, 70, 80, 90, 100, 10, 20, 30, 40],
       [60, 70, 80, 90, 100, 10, 20, 30, 40, 50],
       [70, 80, 90, 100, 10, 20, 30, 40, 50, 60],
       [80, 90, 100, 10, 20, 30, 40, 50, 60, 70],
       [90, 100, 10, 20, 30, 40, 50, 60, 70, 80],
       [100, 10, 20, 30, 40, 50, 60, 70, 80, 90]
     ]

# linear variable cost: the cost of transporting one unit of gas
vc = 1

# unsatisfied demand: penalty for each unit of gas which is not consumed or produced
p = 100

# linear fixed cost: the cost of opening an arc is proportional to its capacity
fc = [ [10 * c for c in row] for row in ca]



### Here is the MIP formulation

|           |              |
| ----------|:-------------|
| $d_i$  |  gas demand at node $i$  |
| $gmax_i$ | maximum gas production at node $i$ |
| $vc$  | cost of transporting one unit of gas through a pipeline  |
| $fc_{ij}$  | cost of opening pipeline $(i,j)$  |
| $ca_{i,j}$  |  capacity of pipeline $(i,j)$  |
| $p$  | penalty for each unit of gas which is not consumed or produced  |
| $g_i$ | gas produced at node $i$ |
| $x_{i,j}$  | volume of gas going through pipeline $(i,j)$  |
| $y_{ij}$  | bynary decision, =1 if pipeline $(i,j)$ is open, 0 otherwise  |
| $z_i$  | demand which is not fulfilled at node $i$ |

\begin{align*}
\min & \sum_{i,j} vc*x_{i,j} + \sum_{i,j} fc_{i,j}*y_{i,j} + \sum_i p*z_i & \\
\text{subject to:} && \\
& \sum_j x_{j,i} + g_i = \sum_j x_{j,i} + d_i - z_i & \forall i \\
& x_{i,j} \leq ca_{i,j} * y_{i,j} & \forall i,j \\
& g_i \leq gmax_i & \forall i \\
& g_i, x_{i,j}, z_i \geq 0 & \forall i,j 
& y_{i,j} \in \mathbb{B}
\end{align*}

#### 2. Formulate the Benders' master problem and sub-problem

##### Master:

\begin{align*}
\min & \sum_{i,j} fc_{i,j}*y_{i,j} + \theta & \\
\text{subject to:} && \\
& \theta \geq \sum_i d_i * \alpha^n_i +  \sum_{i,j} ca_{i,j}*\beta^n_{i,j}*y_{i,j}  & \forall i \\
& y_{i,j} \in \mathbb{B} & \forall i,j 
\end{align*}

##### Sub-problem:

\begin{align*}
\min & \sum_{i,j} vc*x_{i,j} + \sum_i p*z_i & \\
\text{subject to:} && \\
& \sum_j x_{j,i} + g_i = \sum_j x_{j,i} + d_i - z_i & (\alpha^n_i) \quad \forall i \\
& x_{i,j} \leq ca_{i,j}*y^n_{i,j} & (\beta^n_{i,j}) \quad \forall i,j \\
& x_{i,j}, z_i \geq 0 & \forall i,j 
\end{align*}

#### 3. Implement the Benders' decompostion

#### First, implement the subproblem.

In [3]:
from docplex.mp.model import Model

def benders_subproblem(y):
    mdl = Model(name='sub_problem')
    x = mdl.continuous_var_matrix(N, N, lb=0, name='x')
    g = mdl.continuous_var_list(N,lb=0,ub=gmax,name='g')
    z = mdl.continuous_var_list(N, lb=0, name='z')
    
    flow = [ 0 for i in NODES]
    open_arcs = [ [0 for j in NODES] for i in NODES]
    
    for i in NODES:
        flow[i] = mdl.add_constraint(
            sum(x[j,i] for j in NODES) + g[i] == sum(x[i,j] for j in NODES) + d[i] - z[i],'flow_%d' % i)
    
    for i in NODES:
        for j in NODES:
            open_arcs[i][j] = mdl.add_constraint(
                x[i, j] <= ca[i][j] * y[i][j], 'open_arc_%d_%d' %(i,j))

    mdl.minimize(vc * sum(x[i, j] for j in NODES for i in NODES) + p * sum(z))
    mdl.solve()
    
    const = sum(d[i] * flow[i].dual_value for i in NODES)
    coeffs = [[ca[i][j] * open_arcs[i][j].dual_value for j in NODES] for i in NODES]
    
    return (mdl.objective_value,  # the objective value
            const,  # the constant part of the benders' cut
            coeffs)  # the coefficients of the benders' cut

In [4]:
benders_subproblem([[0]*N]*N)

(50000.0,
 50000.0,
 [[0,
   -1980.0,
   -2970.0,
   -3960.0,
   -4950.0,
   -5940.0,
   -6930.0,
   -7920.0,
   -8910.0,
   -9900.0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

#### Second, implement the master problem

Create a master problem without any cut and solve it.

In [5]:
master = Model(name='master')
y = [ [master.binary_var(name='y_%d_%d' %(i,j)) for j in NODES] for i in NODES]
theta = master.continuous_var(lb=0, name='theta')
master.minimize(sum(fc[i][j] * y[i][j] for i in NODES for j in NODES) + theta)
master.solve()
print('Solution cost: %.2f' % master.objective_value)

Solution cost: 0.00


#### Then, create the resolution loop that will:
1. Solve the sub-problem with a solution
2. Add the benders' cut to the master problem
3. Solve the master problem

In [6]:
master = Model(name='master')
y = [ [master.binary_var(name='y_%d_%d' %(i,j)) for j in NODES] for i in NODES]
theta = master.continuous_var(lb=0, name='theta')
master.minimize(sum(fc[i][j] * y[i][j] for i in NODES for j in NODES) + theta)

iteration = 0
LB = 0
UB = p * sum(abs(d[i]) for i in NODES)

while True:
    # 1. Solve the master
    master.solve()
    LB = master.objective_value
    
    # 2. Solve the subproblem
    y_sol = [[y[i][j].solution_value for j in NODES] for i in NODES]
    obj, const, coeffs = benders_subproblem(y_sol)

    # Update the UB
    UB = obj + sum(fc[i][j] * y_sol[i][j] for i in NODES for j in NODES)
    print('Iter=%d; LB=%.2f; UB=%.2f' %(iteration, LB, UB))
    
    # check if you have guess theta correctly
    if abs(UB - LB) < 1e-5:
        break
    
    # 3. Add the benders' cut to the master problem
    master.add_constraint(theta >= const + sum(coeffs[i][j] * y[i][j] for i in NODES for j in NODES),
                         'bender_cut_%d' % iteration)
    iteration += 1

print(''.join(['=']*40))
for p in [(i,j) for i in NODES for j in NODES if y[i][j].solution_value]:
    print('Arc %d to %d is open.' % p)

Iter=0; LB=0.00; UB=50000.00
Iter=1; LB=5100.00; UB=25697.00
Iter=2; LB=7200.00; UB=22083.00
Iter=3; LB=7200.00; UB=14537.00
Iter=4; LB=7200.00; UB=17967.00
Iter=5; LB=7300.00; UB=18087.00
Iter=6; LB=7300.00; UB=15323.00
Iter=7; LB=7400.00; UB=13267.00
Iter=8; LB=7500.00; UB=18199.00
Iter=9; LB=7500.00; UB=15033.00
Iter=10; LB=7500.00; UB=12877.00
Iter=11; LB=7600.00; UB=10243.00
Iter=12; LB=7700.00; UB=12019.00
Iter=13; LB=7800.00; UB=14667.00
Iter=14; LB=7800.00; UB=12903.00
Iter=15; LB=7800.00; UB=11903.00
Iter=16; LB=8000.00; UB=10947.00
Iter=17; LB=8000.00; UB=12123.00
Iter=18; LB=8000.00; UB=14553.00
Iter=19; LB=8000.00; UB=8673.00
Iter=20; LB=8273.00; UB=15837.00
Iter=21; LB=8473.00; UB=12264.00
Iter=22; LB=8573.00; UB=10553.00
Iter=23; LB=8573.00; UB=11043.00
Iter=24; LB=8573.00; UB=9598.00
Iter=25; LB=8573.00; UB=10068.00
Iter=26; LB=8573.00; UB=13511.00
Iter=27; LB=8573.00; UB=14463.00
Iter=28; LB=8573.00; UB=9117.00
Iter=29; LB=8593.00; UB=9279.00
Iter=30; LB=8673.00; UB=867